## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np

from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import Model

tf.keras.backend.clear_session()

## Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/)

[tf.keras.datasets.mnist](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/mnist/load_data) returns: ***Tuple of Numpy arrays: (x_train, y_train), (x_test, y_test).***

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

Use tf.data to batch and shuffle the dataset.

from_tensor_slices

In [3]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## Defining the Model

In [29]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(8, 3, activation='relu')
    self.pool = MaxPooling2D()
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x0 = self.conv1(x)
    x1 = self.pool(x0)
    x2 = self.flatten(x1)
    x3 = self.d1(x2)
    return self.d2(x3)


Choose an optimizer and loss function for training:

In [43]:
# see
# https://github.com/tensorflow/tensorflow/blob/e5bf8de410005de06a7ff5393fafdf832ef1d4ad/tensorflow/python/keras/utils/layer_utils.py#L224-L228
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer
def print_model_summary(model, input_shape):
  input_shape_nobatch = input_shape[1:]
  model.build(input_shape)
  inputs = tf.keras.Input(shape=input_shape_nobatch)
  
  if not hasattr(model, 'call'):
      raise AttributeError("User should define 'call' method in sub-class model!")
  
  _ = model.call(inputs)
  #model.layers[0].output_shape
  #model.layers[0].count_params()
  #
  model.summary()

print_model_summary(model, x_train.shape)

Model: "my_model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 26, 26, 8)         80        
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 13, 13, 8)         0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 1352)              0         
_________________________________________________________________
dense_24 (Dense)             (None, 128)               173184    
_________________________________________________________________
dense_25 (Dense)             (None, 10)                1290      
Total params: 174,554
Trainable params: 174,554
Non-trainable params: 0
_________________________________________________________________


In [5]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [6]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [7]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

Test the model:

In [8]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

## Training step

In [9]:
EPOCHS = 8

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 0.19169411063194275, Accuracy: 94.48333740234375, Test Loss: 0.07949524372816086, Test Accuracy: 97.47999572753906
Epoch 2, Loss: 0.06467931717634201, Accuracy: 98.01333618164062, Test Loss: 0.07282432168722153, Test Accuracy: 97.57999420166016
Epoch 3, Loss: 0.04289546608924866, Accuracy: 98.68500518798828, Test Loss: 0.05411408841609955, Test Accuracy: 98.11000061035156
Epoch 4, Loss: 0.028869809582829475, Accuracy: 99.09666442871094, Test Loss: 0.04795987159013748, Test Accuracy: 98.45

## Testing the Model

In [10]:
test_image, test_label = next(test_ds.__iter__())
test_image = tf.slice(test_image, [0,0,0,0], [1,-1,-1,-1])
prediction = model(test_image, training=False)
print("label: ", test_label[0])
print("pred:  ", tf.math.argmax(prediction[0]))

label:  tf.Tensor(7, shape=(), dtype=uint8)
pred:   tf.Tensor(7, shape=(), dtype=int64)


In [11]:
model.save('saved_model/test_model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/test_model/assets
